In [3]:
import autokeras as ak
import pandas as pd
import numpy as np
df_train = pd.read_csv('./titanic/train.csv')
df_test = pd.read_csv("./titanic/test.csv")

train_test_data = [df_train, df_test]  # combining train and test dataset

title_mapping = {"Mr": "Mr", "Miss": "Miss", "Mrs": "Mrs",
                 "Master": "Boy", "Dr": "Gentleman", "Rev": "Gentleman", "Col": "unknown", "Major": "Gentleman", "Mlle": "unknown", "Countess": "Lady",
                 "Ms": "Miss", "Lady": "Lady", "Jonkheer": "unknown", "Don": "Gentleman", "Dona": "Lady", "Mme": "unknown", "Capt": "Gentleman", "Sir": "Gentleman"}
# features_drop = ['PassengerId', 'Ticket', 'Name', 'Title', 'Cabin', 'Family']
features_drop = ['PassengerId', 'Ticket', 'Name', 'Family']

# create ageset
concatset = (pd.concat([df_train, df_test]))
concatset['Title'] = concatset['Name'].str.extract(
    ' ([A-Za-z]+)\.', expand=False)
concatset['Title'] = concatset['Title'].map(title_mapping)
ageset = concatset.groupby(['Title'])['Age'].mean()
fareset = concatset.groupby(['Pclass'])['Fare'].mean()

for dataset in train_test_data:
    # mapping
    dataset['Title'] = dataset['Name'].str.extract(
        ' ([A-Za-z]+)\.', expand=False)
    dataset["Cabin"] = dataset["Cabin"].str[:1]

    # fillna
    # dataset["Age"].fillna(dataset.groupby(
    #     "Title")["Age"].transform("median"), inplace=True)
    dataset['Embarked'] = dataset['Embarked'].fillna('unknown')
    dataset['Cabin'] = dataset['Cabin'].fillna('unknown')
    dataset["Fare"].fillna(dataset["Pclass"].map(fareset), inplace=True)
    # dataset["Fare"].fillna(dataset.groupby("Pclass")[
    #                        "Fare"].transform("median"), inplace=True)

    dataset["Age"].fillna(dataset["Title"].map(ageset), inplace=True)
    # add
    dataset["Family"] = dataset["SibSp"] + dataset["Parch"]
    dataset['Alone'] = np.where(dataset['Family'] > 0, 'n', 'y')

    # delete
    dataset.drop(features_drop, axis=1, inplace=True)

In [2]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       886 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Cabin     891 non-null    object 
 8   Embarked  891 non-null    object 
 9   Title     891 non-null    object 
 10  Alone     891 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 76.7+ KB
